# Final Project (Dennis Ong 27877331)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from camutils import *
from meshutils import writeply

import cv2
import pickle
import camutils
import scipy

%matplotlib inline

## Calibrating Intrinsic Parameters

In [2]:
calibration = pickle.load(open('/home/dennis/Documents/CS117/FinalProject/calibration.pickle', 'rb'))

# individual intrinsic parameters
fx = calibration['fx']
fy = calibration['fy']
cx = calibration['cx']
cy = calibration['cy']

# estimated radial distortion coefficients
dist = calibration['dist']

## Calculating Extrinsic Parameters

In [3]:
fl = (fx + fy) / 2
offset = np.array([[cx, cy]]).T
camL = Camera(f = fl, c = offset, t = np.array([[0, 0, 0]]).T, R = makerotation(0, 0, 0))
camR = Camera(f = fl, c = offset, t = np.array([[0, 0, 0]]).T, R = makerotation(0, 0, 0))

# find left camera 2-d coordinates
imgL = plt.imread('/home/dennis/Documents/CS117/FinalProject/calib/frame_C1_01.jpg')
ret, cornersL = cv2.findChessboardCorners(imgL, (8, 6), None)
pts2L = cornersL.squeeze().T

# find right camera 2-d coordinates
imgR = plt.imread('/home/dennis/Documents/CS117/FinalProject/calib/frame_C0_01.jpg')
ret, cornersR = cv2.findChessboardCorners(imgR, (8, 6), None)
pts2R = cornersR.squeeze().T

# generating 3-d coordinates in cm
pts3 = np.zeros((3, 6 * 8))
yy, xx = np.meshgrid(np.arange(8), np.arange(6))
pts3[0,:] = 2.8 * xx.reshape(1, -1)
pts3[1,:] = 2.8 * yy.reshape(1, -1)

# calibrating pose with assignment #3 parameters
camLParams = [0, -30, 0, 0, 0, -15]
camRParams = [0, 30, 0, 0, 0, -15]
camL = calibratePose(pts3, pts2L, camL, camLParams)
camR = calibratePose(pts3, pts2R, camR, camRParams)

## New Reconstruction Code

In [4]:
def reconstructNew(colorL, colorR, bgL, bgR, imprefixL, imprefixR, thresh, threshC, camL, camR):
    """
    Simple reconstruction based on triangulating matched pairs of points
    between to view which have been encoded with a 20bit gray code.
    
    CHANGES: (1) Added color and background image masking
             (2) Recording of corresponding pixel color

    Parameters
    ----------
    color : str
      location where object color images are stored
      
    bg : str
      location for where background color images are stored
      
    imprefix : str
      prefix for where the images are stored

    thresh : float
      decodability threshold
      
    thresh : float
      color - background threshold

    camL, camR : Camera
      camera parameters

    Returns
    -------
    pts2L,pts2R : 2D numpy.array (dtype=float)

    pts3 : 2D numpy.array (dtype=float)
    
    color2L : 2D numpy.array (dtype=)

    """
    
    backgroundL = cv2.imread(bgL)
    backgroundR = cv2.imread(bgR)
    objectL = cv2.imread(colorL)
    objectR = cv2.imread(colorR)

    CLh, maskLh = decode(imprefixL, 0, thresh)
    CLv, maskLv = decode(imprefixL, 20, thresh)
    CRh, maskRh = decode(imprefixR, 0, thresh)
    CRv, maskRv = decode(imprefixR, 20, thresh)
    
    CL = CLh + 1024 * CLv
    maskL = maskLh * maskLv
    CR = CRh + 1024 * CRv
    maskR = maskRh * maskRv
    
    # Added code here for difference between object and background for masking
    
    maskLDiff = objectL - backgroundL # Subtraction between each "point" (R, G, B)
    maskLDiff = maskLDiff * maskLDiff # Squaring of each result
    maskLDiff = np.sum(maskLDiff, axis = 2) # Adding all squared results
    maskLDiff = np.sqrt(maskLDiff) # Rooting to find distance
    
    maskRDiff = objectR - backgroundR # Same steps as left images
    maskRDiff = maskRDiff * maskRDiff
    maskRDiff = np.sum(maskRDiff, axis = 2)
    maskRDiff = np.sqrt(maskRDiff)
    
    threshIndicesL = maskLDiff < threshC
    maskLDiff[threshIndicesL] = 0
    
    threshIndicesR = maskRDiff < threshC
    maskRDiff[threshIndicesR] = 0
    
    maskL = maskL * maskLDiff
    maskR = maskR * maskRDiff

    # End code here
    
    h = CR.shape[0]
    w = CR.shape[1]

    subR = np.nonzero(maskR.flatten())
    subL = np.nonzero(maskL.flatten())

    CRgood = CR.flatten()[subR]
    CLgood = CL.flatten()[subL]

    _,submatchR,submatchL = np.intersect1d(CRgood,CLgood,return_indices=True)

    matchR = subR[0][submatchR]
    matchL = subL[0][submatchL]

    xx,yy = np.meshgrid(range(w),range(h))
    xx = np.reshape(xx,(-1,1))
    yy = np.reshape(yy,(-1,1))

    pts2R = np.concatenate((xx[matchR].T,yy[matchR].T),axis=0)
    pts2L = np.concatenate((xx[matchL].T,yy[matchL].T),axis=0)

    pts3 = triangulate(pts2L,camL,pts2R,camR)
    
    # Added code here to save RGB values
    
    colorL = objectL[pts2L[1], pts2L[0]].T
    colorR = objectR[pts2R[1], pts2R[0]].T
    color = (colorL + colorR) / 2
    color = color / 255
    color[[0, 2]] = color[[2, 0]]
    
    # End code here

    return pts2L, pts2R, pts3, color

## Mesh Function

In [5]:
def createMesh(threshTri, bounding, pts2L, pts2R, pts3, colors):
    # Dropping outside bounding box
    goodPoints = np.nonzero((pts3[0,:] > bounding[0]) & (pts3[0,:] < bounding[1]) &
                            (pts3[1,:] > bounding[2]) & (pts3[1,:] < bounding[3]) &
                            (pts3[2,:] > bounding[4]) & (pts3[2,:] < bounding[5]))
    pts3 = pts3[:,goodPoints[0]]
    pts2L = pts2L[:,goodPoints[0]]
    pts2R = pts2R[:,goodPoints[0]]
    colors = colors[:,goodPoints[0]]
    
    # Computing initial triangulation
    Triangles =  scipy.spatial.Delaunay(pts2L.T)
    tri = Triangles.simplices
    
    ## Creating triangle
    side1 = np.sqrt(np.sum(np.power(pts3[:,tri[:,0]] - pts3[:,tri[:,1]], 2), axis = 0))
    side2 = np.sqrt(np.sum(np.power(pts3[:,tri[:,0]] - pts3[:,tri[:,2]], 2), axis = 0))
    side3 = np.sqrt(np.sum(np.power(pts3[:,tri[:,1]] - pts3[:,tri[:,2]], 2), axis = 0))
    
    ## Thresholding triangle edges
    goodTri = (side1 < threshTri) & (side2 < threshTri) & (side3 < threshTri)
    tri = tri[goodTri,:]
    
    return pts2L, pts2R, pts3, colors, tri  

## Reconstruct Mesh (combination) Function

In [6]:
def reconstructMesh(fileName, colorL, colorR, bgL, bgR, imprefixL, imprefixR, 
                    thresh, threshC, threshTri, bound, camL, camR):
    pts2L, pts2R, pts3, colors = reconstructNew(objectL, objectR, backgroundL, backgroundR, 
                                           prefixL, prefixR, thresh, threshC, camL, camR)
    pts2L, pts2R, pts3, colors, tri = createMesh(threshTri, bound, pts2L, pts2R, pts3, colors) 
    writeply(pts3, colors, tri, fileName)

## Running reconstructMesh with all sets of images

In [7]:
root = '/home/dennis/Documents/CS117/FinalProject/teapot/'

# First set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([0, 19, 1, 22, 17, 27]) 
backgroundL, backgroundR = root + 'grab_0_u/color_C1_00.png', root + 'grab_0_u/color_C0_00.png'
objectL, objectR         = root + 'grab_0_u/color_C1_01.png', root + 'grab_0_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_0_u/frame_C1_', root + 'grab_0_u/frame_C0_'
reconstructMesh('set01.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)
# Second set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([-3, 18, 4, 18, 18, 28])
backgroundL, backgroundR = root + 'grab_1_u/color_C1_00.png', root + 'grab_1_u/color_C0_00.png'
objectL, objectR         = root + 'grab_1_u/color_C1_01.png', root + 'grab_1_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_1_u/frame_C1_', root + 'grab_1_u/frame_C0_'
reconstructMesh('set02.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)

# Third set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([-3, 18, -1, 21, 17, 26])
backgroundL, backgroundR = root + 'grab_2_u/color_C1_00.png', root + 'grab_2_u/color_C0_00.png'
objectL, objectR         = root + 'grab_2_u/color_C1_01.png', root + 'grab_2_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_2_u/frame_C1_', root + 'grab_2_u/frame_C0_'
reconstructMesh('set03.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)

# Fourth set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([-3, 18, 5, 18, 14, 24])
backgroundL, backgroundR = root + 'grab_3_u/color_C1_00.png', root + 'grab_3_u/color_C0_00.png'
objectL, objectR         = root + 'grab_3_u/color_C1_01.png', root + 'grab_3_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_3_u/frame_C1_', root + 'grab_3_u/frame_C0_'
reconstructMesh('set04.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)

# Fifth set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([-3, 18.5, -7, 23, 14, 25])
backgroundL, backgroundR = root + 'grab_4_u/color_C1_00.png', root + 'grab_4_u/color_C0_00.png'
objectL, objectR         = root + 'grab_4_u/color_C1_01.png', root + 'grab_4_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_4_u/frame_C1_', root + 'grab_4_u/frame_C0_'
reconstructMesh('set05.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)

# Sixth set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([6, 19, 3, 24, 11, 26])
backgroundL, backgroundR = root + 'grab_5_u/color_C1_00.png', root + 'grab_5_u/color_C0_00.png'
objectL, objectR         = root + 'grab_5_u/color_C1_01.png', root + 'grab_5_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_5_u/frame_C1_', root + 'grab_5_u/frame_C0_'
reconstructMesh('set06.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)

# Seventh set
thresh, threshC, threshTri = 0.02, 0.1, 1.0
teapotBound = np.array([6, 17, 7, 21, 19, 24])
backgroundL, backgroundR = root + 'grab_6_u/color_C1_00.png', root + 'grab_6_u/color_C0_00.png'
objectL, objectR         = root + 'grab_6_u/color_C1_01.png', root + 'grab_6_u/color_C0_01.png'
prefixL, prefixR         = root + 'grab_6_u/frame_C1_', root + 'grab_6_u/frame_C0_'
reconstructMesh('set07.ply', objectL, objectR, backgroundL, backgroundR, prefixL, prefixR,
                thresh, threshC, threshTri, teapotBound, camL, camR)

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )